In [4]:
!pip install xarray

    100% |████████████████████████████████| 419kB 1.4MB/s eta 0:00:01
You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import xarray as xr

In [7]:
import numpy as np
import pandas as pd
import datetime as dt

import pathlib
import multiprocessing, subprocess
import urllib.request
from urllib.request import Request, urlopen
from urllib.error import URLError

In [16]:
vapour_url = "http://www.bom.gov.au/web03/ncc/www/awap/vprp/vprph15/daily/grid/0.05/history/nat/"

In [17]:
url = vapour_url + "2018010120180101.grid.Z"

In [18]:
archive_file = "VP3pm_20180101.grid.Z"

In [19]:
try:
    urllib.request.urlretrieve(url, archive_file)
except URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)

In [22]:
!pip install netcdf4

    100% |████████████████████████████████| 6.3MB 107kB/s eta 0:00:01    91% |█████████████████████████████▍  | 5.8MB 36.9MB/s eta 0:00:01
You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
subprocess.run(["uncompress", "-k", archive_file], stdout=subprocess.PIPE)

CompletedProcess(args=['uncompress', '-k', 'VP3pm_20180101.grid.Z'], returncode=0, stdout=b'')

In [28]:
!conda install -y rasterio

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    affine:        2.2.0-py_0        conda-forge
    attrs:         17.4.0-py_0       conda-forge
    boost:         1.66.0-py36_1     conda-forge
    boto3:         1.6.7-py_0        conda-forge
    botocore:      1.9.7-py_0        conda-forge
    click-plugins: 1.0.3-py36_0      conda-forge
    cligj:         0.4.0-py36_0      conda-forge
    docutils:      0.14-py36_0       conda-forge
    freexl:        1.0.5-0           conda-forge
    gdal:          2.2.4-py36_0      conda-forge
    geos:          3.6.2-1           conda-forge
    geotiff:       1.4.2-1           conda-forge
    giflib:        5.1.4-0           conda-forge
    hdf4:          4.2.13-0          conda-forge
    jmespath:      0.9.3-py36_0      conda-forge
    json-c:        0.12.1-0          conda-forge
    kealib:        1.4.7-4          

In [43]:
y = 2018
m = 1
d = 1

with xr.open_rasterio("VP3pm_20180101.grid") as src:
    print(src)
    src.name="VP3pm"
    print(src.attrs)
    src.to_netcdf("VP3pm_20180101.nc")

<xarray.DataArray (band: 1, y: 691, x: 886)>
[612226 values with dtype=float32]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 -10.0 -10.05 -10.1 -10.15 -10.2 -10.25 -10.3 -10.35 ...
  * x        (x) float64 112.0 112.0 112.1 112.1 112.2 112.2 112.3 112.3 ...
Attributes:
    transform:   (111.975, 0.05, 0.0, -9.974999999999994, 0.0, -0.05)
    res:         (0.05, 0.05)
    is_tiled:    0
    nodatavals:  (99999.8984375,)
OrderedDict([('transform', (111.975, 0.05, 0.0, -9.974999999999994, 0.0, -0.05)), ('res', (0.05, 0.05)), ('is_tiled', 0), ('nodatavals', (99999.8984375,))])


In [62]:
with xr.open_dataset("VP3pm_20180101.nc") as src:
    src = src.rename({'y':'latitude', 'x':'longitude', 'band':'time'})
    src.coords['time'] = [dt.datetime(int(y), int(m), int(d))]
    src.attrs['time:units'] = "Days since %s-%s-%s 00:00:00" % (y,m,d)
    src.attrs['crs'] = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs "
    print(src)
    src.to_netcdf("VP3pm_20180101_nc.nc")
    src.close()

<xarray.Dataset>
Dimensions:    (latitude: 691, longitude: 886, time: 1)
Coordinates:
  * time       (time) datetime64[ns] 2018-01-01
  * latitude   (latitude) float64 -10.0 -10.05 -10.1 -10.15 -10.2 -10.25 ...
  * longitude  (longitude) float64 112.0 112.0 112.1 112.1 112.2 112.2 112.3 ...
Data variables:
    VP3pm      (time, latitude, longitude) float32 ...
Attributes:
    time:units:  Days since 2018-1-1 00:00:00
    crs:         +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs 


In [63]:
src = xr.open_dataset("VP3pm_20180101_nc.nc")
print(src)

<xarray.Dataset>
Dimensions:    (latitude: 691, longitude: 886, time: 1)
Coordinates:
  * time       (time) datetime64[ns] 2018-01-01
  * latitude   (latitude) float64 -10.0 -10.05 -10.1 -10.15 -10.2 -10.25 ...
  * longitude  (longitude) float64 112.0 112.0 112.1 112.1 112.2 112.2 112.3 ...
Data variables:
    VP3pm      (time, latitude, longitude) float32 ...
Attributes:
    time:units:  Days since 2018-1-1 00:00:00
    crs:         +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs 


In [57]:
src.isel(time=0)["VP3pm"]

<xarray.DataArray 'VP3pm' (latitude: 691, longitude: 886)>
[612226 values with dtype=float32]
Coordinates:
    time       datetime64[ns] 2018-01-01
  * latitude   (latitude) float64 -10.0 -10.05 -10.1 -10.15 -10.2 -10.25 ...
  * longitude  (longitude) float64 112.0 112.0 112.1 112.1 112.2 112.2 112.3 ...
Attributes:
    transform:   [  1.11975000e+02   5.00000000e-02   0.00000000e+00  -9.9750...
    res:         [ 0.05  0.05]
    is_tiled:    0
    nodatavals:  99999.8984375

In [8]:
import asyncio
from pprint import pprint

import random


async def coro(tag):
    print(">", tag)
    await asyncio.sleep(random.uniform(1, 3))
    print("<", tag)
    return tag


loop = asyncio.get_event_loop()

group1 = asyncio.gather(*[coro("group 1.{}".format(i)) for i in range(1, 6)])
group2 = asyncio.gather(*[coro("group 2.{}".format(i)) for i in range(1, 4)])
group3 = asyncio.gather(*[coro("group 3.{}".format(i)) for i in range(1, 10)])

all_groups = asyncio.gather(group1, group2, group3)

results = loop.run_until_complete(all_groups)

loop.close()

pprint(results)

RuntimeError: This event loop is already running

> group 1.5
> group 1.2
> group 1.3
> group 1.1
> group 1.4
> group 2.2
> group 2.3
> group 2.1
> group 3.2
> group 3.9
> group 3.3
> group 3.8
> group 3.6
> group 3.7
> group 3.1
> group 3.5
> group 3.4
< group 1.1
< group 3.4
< group 2.3
< group 1.3
< group 3.6
< group 3.7
< group 3.9
< group 1.5
< group 2.2
< group 3.8
< group 3.1
< group 3.3
< group 3.5
< group 1.2
< group 1.4
< group 3.2
< group 2.1
